## SET ENVIRONMENT

In [1]:
import os 
import pandas as pd 
import re

os.chdir(r'C:\Users\Adam\Python Scripts\feldolgozott_kepek')
mapping = pd.read_excel(r'C:\Users\Adam\Python Scripts\invoice_mapping\invoice_item_map.xlsx')

## ITEMIZE INVOICE

In [2]:
# SUBSET MAPPING 
unique_types = list(set(mapping['Type']))

itemized = pd.DataFrame()
itemized['Field_Name'] = []
itemized['File_Name'] = []
itemized['Value'] = []

app1 = pd.Series()
app2 = pd.Series()
app3 = pd.Series()

C:\Users\Adam\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
C:\Users\Adam\anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Adam\anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()


In [3]:

## LOOP I  - go through each Invoice Type ######################################
for m in range(len(unique_types)):
    
    # SUBSET MAPPING PER TYPE 
    
    # printings 
    print('########################   1 - TYPE RUNNING:   ################################')
    print(unique_types[m])
    print('WORKDIR SET TO:')
    
    # subset mapping 
    applicable_mapping = mapping.loc[mapping['Type'] == unique_types[m]]
    # setting wd for each mapping
    dry = r'C:\Users\Adam\Python Scripts\feldolgozott_kepek' + '\\' + unique_types[m] +'\\' + '_szovegge_alakitott_kepek_'
    os.chdir(dry)

    
    
    ## LOOP II  - go thorugh each field name ######################################
    for f in range(len(applicable_mapping['Field_Name'])):
        
        # printings 
        print(' ############################### 2 - FIELD NAME RUNNING ###############################:')
        print(applicable_mapping['Field_Name'][f])
        
        # file variables 
        # coordinate column
        if type(applicable_mapping['Coordinate_col'].tolist()[f]) == int: 
            coordinate_col = applicable_mapping['Coordinate_col'][f] -1 
        else: 
            coordinate_col = applicable_mapping['Coordinate_col'][f] 
        
        
        # Coordinate row 
        separator = applicable_mapping['Separator'][f]
        coordinate_row = applicable_mapping['Coordinate_row'][f]
        field = applicable_mapping['SearchFor'][f]
        remove = applicable_mapping['Remove'][f]
        text_files = os.listdir()
        # pre-create field name for next loop 
        app1_1 = pd.Series(applicable_mapping['Field_Name'][f])
        


        
        ## LOOP III - go through each file  ######################################
        for f in range(len(text_files)):
            
    
            # prints
            print(' ############################### 3 - RUNNING FILE: ###############################')
            print(text_files[f])
            
            # create append variable for running text file name 
            app2_2 = pd.Series(text_files[f])
            app2 = app2.append(app2_2)
            app1 = app1.append(app1_1)
            
            
            # READ IN FILE 
            text = open(text_files[f]).read()
            
            # LINE NAVIGATION
            # GET LINE NUMBER FOR FIELD TO BE FOUND 
            lookup = field
            with open(text_files[f]) as myFile:
                for num, line in enumerate(myFile, 1):
                    if lookup in line:
                        print('found at line:'), print(num)
                        line_num = num-1 + coordinate_row
                        
            # GET INVOICE ITEM 
            var = text.splitlines()[line_num]
            print(var)
            # REMOVE PREFIX
            var = var.replace(remove, '')

            
            '''  '''
            # COLUMN NAVIGATZION 
            
            if type(coordinate_col.tolist()) == int and coordinate_col >= -1: 
                    var = pd.Series(var)
                    splitted = var.str.split(expand = True, pat = separator)
                    var = splitted[coordinate_col+1]
                    # REMOVE SEPARATOR
                    var = var.replace(separator, '')

            
            
            else: 
                var = pd.Series(var)
                splitted = var.str.split(expand = True, pat = separator )
                splitted = splitted.loc[:, coordinate_col]
                var = splitted
                # REMOVE SEPARATOR
                var = var.replace(separator, '')
            
                
                
            #  APPEND 3
            var = pd.Series(var)
            app3 = app3.append(var)



########################   1 - TYPE RUNNING:   ################################
Type1
WORKDIR SET TO:
 ############################### 2 - FIELD NAME RUNNING ###############################:
Bizonylat Sorszáma 
 ############################### 3 - RUNNING FILE: ###############################
t1_szamla_1.txt
found at line:
8
Bizonylat sorszáma: @ SZ00114/2014
 ############################### 3 - RUNNING FILE: ###############################
t1_szamla_2.txt
found at line:
8
Bizonylat sorszáma: @ 888831234
 ############################### 3 - RUNNING FILE: ###############################
t1_szamla_3.txt
found at line:
8
Bizonylat sorszáma: @ 888831234
 ############################### 3 - RUNNING FILE: ###############################
t1_szamla_4.txt
found at line:
8
Bizonylat sorszáma: @ 888831234
 ############################### 2 - FIELD NAME RUNNING ###############################:
Szállító
 ############################### 3 - RUNNING FILE: ###############################
t1_szamla_1.t

## BRING IT INTO PANDAS DATAFRAME 

In [4]:
app1 = app1.reset_index(drop=True)
app2 = app2.reset_index(drop=True)
app3 = app3.reset_index(drop=True)

df = pd.concat([app1, app2, app3], axis = 1)
df = df.rename(columns={0: 'Számla_tétel', 1: 'File_neve', 2: 'Érték'})

In [5]:
#df.to_excel(r'C:\Users\Adam\Python Scripts\feldolgozott_kepek\konvertalt_szamlak.xlsx')

In [6]:
df.loc[df['File_neve'] == 't1_szamla_1.txt']

In [17]:
os.chdir(r'C:\Users\Adam\Python Scripts\Output')
df.to_excel('invoice_summary.xlsx')

In [19]:
type(df)

pandas.core.frame.DataFrame